In [1]:
import collections
import math
import os
import random

import numpy as np
import pandas as pd
import tensorflow as tf
import  nltk

In [2]:
ppp = nltk.data.load('../Downloads/ppp.txt', encoding='utf8')
words_p = nltk.tokenize.word_tokenize(ppp)[121:]


alw = nltk.data.load('../Downloads/alw.txt', encoding='utf8')
words_a = nltk.tokenize.word_tokenize(alw)[122:]
words = words_a + words_p

In [3]:
vocabulary_size = 10000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
del words_a
del words_p # Hint to reduce memory.

data_index = 0

In [4]:
word_embed = np.load('./embed3.npy')

In [5]:
word_embed = np.concatenate((word_embed, np.zeros((len(word_embed), 1)), 
                np.arange(len(word_embed)).reshape((len(word_embed), 1)))
               , axis=1)

In [6]:
sentences_vec = []
sentences_word  = []
sent_word = []
sent_vec = []
indices = []
for index, word in enumerate(filter(lambda word: word in dictionary, words)):
    if word in ['?','.','!']:
        sent_vec += [word_embed[dictionary[word]]]
        sent_word += [word]
        if len(sent_vec) > 1:
            sentences_vec += [sent_vec]
            sentences_word += [sent_word]
        sent_vec = []
        sent_word = []
    elif word in ['Chapter', 'CHAPTER'] and words[index+1] in ['1','I']:
        indices += [len(sentences_vec)]
    else:
        sent_vec += [word_embed[dictionary[word]]]
        sent_word += [word]

In [7]:
sentences_vec_t = [np.diff(sent, axis=0) for sent in sentences_vec]

words_vec_t = np.array([[[float(dictionary[sent[index]]), 
                         float(dictionary[sent[index+1]])]
                        for index in range(len(sent)-1)] for sent in 
                         sentences_word])


In [9]:
l2 = np.concatenate((sentences_vec_t[0][:, :-2], words_vec_t[0]), axis=1)
for index, sent in enumerate(sentences_vec_t[1:]):
    nsent = np.concatenate((sent[:, :-2], words_vec_t[index+1]), axis=1)
    l2 = np.concatenate((l2, nsent), axis=0)


In [10]:
np.save('./l2_embed3', l2)

In [12]:
l2 = np.load('./l2_embed2.npy')

In [14]:
def find_nearest_vector(array, value, index):
    indices = np.concatenate((np.arange(index),np.arange(index+1, len(array))))
    sq = np.square(array[indices] - value)
    idx = sq.sum(axis=1).argmin()
    return np.square(array[idx] - value).sum(), idx

In [16]:
tl2 = pd.DataFrame(l2).sample(frac=.2).as_matrix()

In [17]:
nn1 = []
for index, word in enumerate(tl2):
    ret = find_nearest_vector(tl2[:,:-2], tl2[index, :-2], index)
    nn1 += [(ret[0], [reverse_dictionary[word[-2]], reverse_dictionary[word[-1]]],
             [reverse_dictionary[tl2[ret[1]][-2]], 
              reverse_dictionary[tl2[ret[1]][-1]]])]
    if index % 100 == 0:
        print(index)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [18]:
nn1.sort()

In [61]:
test2 = pd.DataFrame(nn1)

In [20]:
none = test[1].apply(lambda x: pd.Series(x))
none.set_axis(1, ['f1','f2'])
ntwo = test[2].apply(lambda x: pd.Series(x))
ntwo.set_axis(1, ['t1','t2'])

In [21]:
t2 = pd.concat((none, ntwo), axis=1)

In [22]:
t2['dist'] = test[0]

In [23]:
t2.to_pickle('./nn2')

In [29]:
nn1 = pd.read_pickle('./nn2')

In [24]:
t2.loc[t2.dist > 0].loc[~(t2.f1 == t2.t1)]

,f1,f2,t1,t2,dist
14523,things,to,assisting,to,5.820766e-11
14524,yield,without,never,without,4.656613e-10
14526,nature,had,already,had,1.644366e-09
14528,denial,.,ever,.,1.420267e-08
14530,blind,",",cat,",",1.794251e-08
14531,nor,at,play,at,1.980516e-08
14532,receiving,in,pounds,in,3.195601e-08
14535,good,to,“as,to,5.081529e-08
14536,that,‘Well,your,uncle,5.145557e-08
14537,came,.,speech,.,5.228503e-08


In [27]:
centroids = np.load('./centroidsl2.npy').item()

test = pd.DataFrame(l2).groupby(centroids.labels_)

nearests = test.apply(nearest)

In [114]:
def nearest(cluster):
    cluster = cluster.as_matrix()
    if len(cluster) < 2:
        return 
    nn1 = []
    for index, word in enumerate(cluster):
        ret = find_nearest_vector(cluster[:,:-2], cluster[index, :-2], index)
        nn1 += [[ret[0], reverse_dictionary[word[-2]], reverse_dictionary[word[-1]],
                 reverse_dictionary[cluster[ret[1]][-2]], 
                  reverse_dictionary[cluster[ret[1]][-1]]]]
    return pd.DataFrame(nn1, columns=['dist','f1','f2','t1','t2'])
        


In [117]:
transitions1 = nearests.reset_index()

In [119]:
transitions1

,level_0,level_1,dist,f1,f2,t1,t2
0,0,0,0.000000,from,the,from,the
1,0,1,0.000000,from,the,from,the
2,0,2,0.000000,from,the,from,the
3,0,3,0.000000,from,the,from,the
4,0,4,0.000000,from,the,from,the
5,0,5,0.000000,from,the,from,the
6,0,6,0.000000,from,the,from,the
7,0,7,0.000000,from,the,from,the
8,0,8,0.000000,from,the,from,the
9,0,9,0.000000,from,the,from,the


In [122]:
transitions1 = transitions1.drop('level_1', axis=1)